In [15]:
import spotipy as spot
import pandas as pd
import json

In [16]:
config = json.load(open(file="config.json", encoding="utf-8"))
config

{'CLIENT_ID': '468b8b024bfb41d5b1957dad2afc766a',
 'CLIENT_SECRET': '8827668f8ed64f13bf8c2e83781c3997',
 'REDIRECT_URI': 'http://localhost:8080'}

In [17]:
# Authorization Flow that WORKS
try:
    token = spot.util.prompt_for_user_token(username='frostcow',
        scope=['playlist-modify-public', 'user-read-recently-played'],
        client_id=config["CLIENT_ID"],
        client_secret=config["CLIENT_SECRET"],
        redirect_uri=config["REDIRECT_URI"])
    test_sp = spot.Spotify(auth=token)
except spot.SpotifyOauthError:
    print("User declined")

In [18]:
type(token)

str

In [19]:
from main import get_last_50_songs

songs = get_last_50_songs(test_sp)

In [20]:
songs

,Track ID,Song Name,Album Name,Artist Name,Explicit,Duration,Song Popularity,Album Release Date
0,0PTYTZvNNtlvdOwWRYrbfC,The Sound of Settling,Transatlanticism (10th Anniversary Edition),Death Cab for Cutie,False,132506,58,2003
1,2IESUzlgtpKwpJ3wbi33OU,"Ilai, Eli, A Lie",As It Were,Marietta,False,126805,28,2015-08-18
2,70UEJN6Cx2EdvtSpICEYzR,Racecar Bed,Awkward Pop Songs,Jank,False,222693,45,2016-04-06
3,1CUeuaJVzuqfHI2fhpLRlF,Tidal Waves,Springtime and Blind,Fiddlehead,False,128119,30,2018-04-13
4,21yuHrj83KWSLRsCvDN7Ul,Cool in Motion,Satisfaction,Narrow Head,False,185640,48,2020-08-14
5,6qmFqZpBMbPEDw4vniOdMv,Spit on a Stranger,Terror Twilight,Pavement,False,184293,54,1999-06-08
6,0DmkBOGgEWMbUJlL6VwwF2,Helmet,Gemini Rights,Steve Lacy,True,201546,76,2022-07-15
7,0Xmz2shisTG8peoqnFj5tu,Sarah,Trick,Alex G,False,176146,63,2012-11-05
8,1rqVob8GLwcVIuBQEHA6aV,Be a Toy,Spring Songs,Title Fight,True,184456,36,2013-11-12
9,6chJN3uEWTwhNoYhx11vD3,Interstate 8,Building Nothing Out of Something,Modest Mouse,False,279200,41,2000-01-18
